# TP1 Alejandro balladares 101118

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Punto 7

7 53. Si para un usuario tenemos la cantidad de acciones que realizó para cada tipo de log
y la cantidad de veces que fue bloqueado: ¿Cuál es la acción que más y menos
correlaciona con ser bloqueado? ¿Qué acción correlaciona más con saber algo
(babel>=0) de inglés? (⭐⭐⭐)


In [4]:
id_logs ='1xbwXF1PqfUlCdPUQwdH1WDY6GrogSA3d'
downloaded = drive.CreateFile({'id': id_logs})
downloaded.GetContentFile('logs.csv')

df_logs = pd.read_csv('logs.csv')



id_languages = '1GC17kS3UFwcv4qoV0FkDdGH6eNgDFCR0'
downloaded = drive.CreateFile({'id': id_languages})
downloaded.GetContentFile('languages.csv')

df_languages = pd.read_csv('languages.csv')

In [5]:
df_logs_lite = df_logs.sample(frac=0.5) #tomo una muestra reducida porque sino me quedo sin ram, de todas formas tomo la muestra más grande que pueda
df_logs_lite = df_logs_lite.drop(columns=['item_id','contributor_username','timestamp', 'comment'])
df_logs_lite

,contributor_id,contributor_ip,logtype,action,title
4164538,2172197.0,NaN,newusers,autocreate,Usuario:Jop108
4460953,2365116.0,NaN,newusers,create,Usuario:Yohandri
6222323,3607912.0,NaN,newusers,create,Usuario:MARCELO JOSE AGUSTONI
10157079,5593926.0,NaN,create,create,Doug Marcaida
545381,288800.0,NaN,newusers,create,Usuario:Chonapi
...,...,...,...,...,...
1691561,242533.0,NaN,delete,delete,Discusión:Detective privado
673513,352887.0,NaN,newusers,create,Usuario:Cheek~eswiki
11779499,NaN,190.84.119.148,create,create,Discusión:Ever Given
7828106,4448668.0,NaN,newusers,create,Usuario:Joseluismoralessantiago


In [6]:
#df_aux = df_logs_lite#[['contributor_id','contributor_ip']]
#df_aux = df_aux.dropna(thresh=4) #si la fila tiene menos de 5 elementos no nulos la quita, esto es para quitar los que tengan la ip y el id como nulos
#df_aux

In [7]:
df_logs_lite = df_logs_lite.dropna(thresh=4) #si la fila tiene menos de 5 elementos no nulos la quita, esto es para quitar los que tengan la ip y el id como nulos
df_logs_lite['contributor_id'] = df_logs_lite['contributor_id'].fillna(df_logs_lite['contributor_ip'])
df_logs_lite['contributor_id'] = df_logs_lite['contributor_id'].astype('str')

In [8]:
df_logs_bloqueados = df_logs_lite.loc[((df_logs_lite.action == 'block') | (df_logs_lite.action == 'reblock'))]
df_logs_bloqueados = df_logs_bloqueados.loc[df_logs.title.str.contains('Usuario', na=False)]
df_logs_bloqueados = df_logs_bloqueados.drop(columns=['contributor_id','contributor_ip'])

df_logs_bloqueados['title'] = df_logs_bloqueados['title'].str[8:]
df_logs_bloqueados

,logtype,action,title
665072,block,block,200.83.37.175
3919502,block,block,125.162.119.208
8673983,block,block,200.125.72.162
1308619,block,block,190.37.213.238
3824256,block,block,200.42.178.18
...,...,...,...
8060866,block,block,Wikipedia86000
11405553,block,block,Fedeux3000
3919975,block,block,180.248.15.9
8597008,block,block,185.190.240.47


In [9]:
df_log_cant_acciones = df_logs_lite.groupby(['contributor_id','action'])['action'].agg({'count'}) 
df_log_cant_acciones.columns = ['acciones_realizadas']
df_log_cant_acciones = df_log_cant_acciones.reset_index()
df_log_cant_acciones

,contributor_id,action,acciones_realizadas
0,1.157.179.90,create,1
1,1.163.5.100,create,1
2,1.42.162.236,create,1
3,1.42.20.114,create,1
4,1.46.96.119,create,2
...,...,...,...
3249539,999994.0,create,1
3249540,999995.0,autocreate,1
3249541,999996.0,create,1
3249542,999997.0,create,1


In [10]:
df_ingles = df_languages.loc[df_languages.babel_lang == 'en']
df_ingles.replace(to_replace='N',value='4.5', inplace=True)
df_ingles['babel_level'] = df_ingles['babel_level'].astype('float')

df_ingles['babel_user'] = df_ingles['babel_user'].astype('float') #le agrego el ".0" al final ya que en los otros dataframes los usuarios son float
df_ingles['babel_user'] = df_ingles['babel_user'].astype('str')   #y es más facil cambiarlo acá que en los otros archivos
df_ingles

,babel_user,babel_lang,babel_level
3893,61378.0,en,0.0
3894,136933.0,en,0.0
3895,182072.0,en,0.0
3896,647542.0,en,0.0
3897,845088.0,en,0.0
...,...,...,...
13355,6253125.0,en,4.5
13356,6254900.0,en,4.5
13357,6277026.0,en,4.5
13358,6327824.0,en,4.5


In [11]:
df_bloqueados = df_logs_bloqueados.groupby('title')['title'].agg({'count'})
df_bloqueados = df_bloqueados.reset_index()
df_bloqueados.columns = ['contributor_id', 'cantidad_de_bloqueos']
df_bloqueados['contributor_id'] = df_bloqueados['contributor_id'].astype('str')
df_bloqueados.loc[df_bloqueados.cantidad_de_bloqueos > 2]

,contributor_id,cantidad_de_bloqueos
233,103.47.132.37,4
255,104.230.23.164,3
347,108.194.183.0,4
369,108.52.6.67,4
463,109.232.16.2,5
...,...,...
155906,WndJo,4
156857,Yttqw,4
157143,Zeraaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaafgaggsggasa...,3
157320,ZurdoA,3


In [12]:
df_log_pivot = df_log_cant_acciones.pivot(index='contributor_id', columns='action', values='acciones_realizadas').fillna(0)
df_log_pivot = df_log_pivot.reset_index()
df_log_pivot

action,contributor_id,add,addlink,autocreate,block,byemail,change,create,create2,delete,delete_redir,deleteuser,event,failure,interwiki,merge,modify,move,move_prot,move_redir,new,newusers,patrol,profilesave,protect,reblock,remove,renameuser,restore,review,revision,rights,selfadd,selfremove,send,skipbadns,skipnouser,thank,unblock,undelete,unprotect,unreview,update,upload,usergroups
0,1.157.179.90,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.163.5.100,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.42.162.236,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.42.20.114,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.46.96.119,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3203368,999994.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3203369,999995.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3203370,999996.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3203371,999997.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_final_7 = df_log_pivot.merge(df_ingles, left_on='contributor_id', right_on='babel_user', how='left').fillna(0) 
df_respuesta_7 = pd.merge(df_final_7, df_bloqueados, on='contributor_id', how='left').fillna(0) #sino es outer no joinea nada


df_respuesta_7['contributor_id'] = df_respuesta_7['contributor_id'].fillna(df_respuesta_7['babel_user'])
df_respuesta_7['ingles'] = df_respuesta_7['babel_level'].transform(lambda x: x != 0)

df_respuesta_7 = df_respuesta_7.drop(columns=['babel_user','babel_lang', 'babel_level']) 

df_respuesta_7 = df_respuesta_7.set_index('contributor_id')
df_respuesta_7

,add,addlink,autocreate,block,byemail,change,create,create2,delete,delete_redir,deleteuser,event,failure,interwiki,merge,modify,move,move_prot,move_redir,new,newusers,patrol,profilesave,protect,reblock,remove,renameuser,restore,review,revision,rights,selfadd,selfremove,send,skipbadns,skipnouser,thank,unblock,undelete,unprotect,unreview,update,upload,usergroups,cantidad_de_bloqueos,ingles
contributor_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.157.179.90,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1.163.5.100,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1.42.162.236,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1.42.20.114,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1.46.96.119,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
999995.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
999996.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [20]:
#df_respuesta_7.loc[df_respuesta_7['cantidad_de_bloqueos'] > 0]

In [15]:
respuesta_7  = df_respuesta_7.corr()
respuesta_7

,add,addlink,autocreate,block,byemail,change,create,create2,delete,delete_redir,deleteuser,event,failure,interwiki,merge,modify,move,move_prot,move_redir,new,newusers,patrol,profilesave,protect,reblock,remove,renameuser,restore,review,revision,rights,selfadd,selfremove,send,skipbadns,skipnouser,thank,unblock,undelete,unprotect,unreview,update,upload,usergroups,cantidad_de_bloqueos,ingles
add,1.000000e+00,-6.446491e-07,-0.000514,0.003113,0.007765,-6.913550e-07,6.192011e-04,0.005125,0.009703,0.005176,-5.794525e-07,0.000271,-5.794525e-07,5.673059e-05,8.526290e-03,0.006535,0.001913,0.002485,0.008254,-5.794525e-07,0.000349,0.007326,2.292158e-02,0.005983,0.001247,9.146633e-01,5.571966e-06,0.004644,8.763311e-02,0.002297,0.007327,0.383144,0.401811,1.936547e-02,-5.794525e-07,-5.794525e-07,0.007326,0.004995,9.561749e-01,0.004033,0.013738,0.273676,0.001324,-5.794525e-07,-0.000025,0.005165
addlink,-6.446491e-07,1.000000e+00,-0.000324,-0.000002,-0.000017,-4.143634e-07,3.127773e-05,-0.000036,-0.000003,-0.000004,-3.472947e-07,-0.000001,-3.472947e-07,-6.690864e-07,-9.863832e-07,-0.000003,-0.000002,-0.000002,-0.000006,-3.472947e-07,-0.000068,-0.000003,-5.671300e-07,-0.000002,-0.000001,-6.981991e-07,-3.845004e-07,-0.000003,-1.891324e-06,-0.000001,-0.000002,-0.000009,-0.000004,-5.358874e-07,-3.472947e-07,-3.472947e-07,0.000162,-0.000002,-3.472947e-07,-0.000002,-0.000001,-0.000002,-0.000008,-3.472947e-07,-0.000015,-0.000028
autocreate,-5.136468e-04,-3.240431e-04,1.000000,-0.001561,-0.014187,9.830359e-04,-8.519136e-03,-0.029477,-0.001864,-0.002417,-2.912710e-04,-0.000766,-2.912710e-04,-4.656644e-04,-4.753362e-04,-0.001965,-0.001072,-0.001518,-0.004397,1.071743e-03,-0.056815,-0.001495,-4.756437e-04,-0.001722,-0.001046,-5.855694e-04,-2.831034e-04,-0.001946,-1.107867e-03,-0.001091,-0.001077,-0.005479,-0.002456,6.021471e-04,-2.912710e-04,-2.912710e-04,-0.004086,-0.002015,-2.912710e-04,-0.001574,-0.000186,-0.001420,-0.006797,-2.912710e-04,-0.012378,-0.010361
block,3.112617e-03,-2.192850e-06,-0.001561,1.000000,0.072616,1.162557e-02,4.905375e-02,0.079978,0.695650,0.287789,-1.971077e-06,0.559521,-1.971077e-06,8.555460e-02,2.006698e-01,0.723783,0.052640,0.236891,0.343455,-1.971077e-06,0.002471,0.171163,-3.218756e-06,0.688401,0.715284,1.701501e-02,2.433993e-03,0.556164,5.400829e-03,0.667062,0.486385,0.008935,0.008258,4.765927e-02,-1.971077e-06,-1.971077e-06,0.165816,0.726423,-1.971077e-06,0.319065,0.000659,0.014007,0.030273,5.360009e-02,-0.000084,0.021831
byemail,7.764857e-03,-1.721746e-05,-0.014187,0.072616,1.000000,-1.846489e-05,2.000572e-03,0.040334,0.060076,0.009149,-1.547617e-05,0.004174,-1.547617e-05,5.121848e-04,8.784939e-02,0.042185,0.002160,0.007850,0.010526,-1.547617e-05,-0.002783,0.011422,-2.527248e-05,0.049537,0.023578,9.301019e-03,2.866830e-05,0.021793,3.586870e-02,0.031291,0.049958,0.010974,0.011292,1.527798e-01,-1.547617e-05,-1.547617e-05,0.016699,0.046552,-1.547617e-05,0.030295,-0.000060,0.010048,0.003123,-1.547617e-05,-0.000658,0.007478
change,-6.913550e-07,-4.143634e-07,0.000983,0.011626,-0.000018,1.000000e+00,1.412211e-03,0.012059,0.008450,0.009716,-3.724568e-07,0.002753,-3.724568e-07,-7.175628e-07,-1.057848e-06,0.027990,0.003371,0.026942,0.010314,9.761871e-01,-0.000073,0.000943,-6.082195e-07,0.007931,0.020568,-7.487848e-07,6.323205e-04,0.004476,-2.028353e-06,0.000347,0.009803,-0.000010,-0.000004,-5.747134e-07,-3.724568e-07,-3.724568e-07,0.022575,0.015892,-3.724568e-07,0.006551,-0.000001,-0.000002,0.000626,2.169302e-01,-0.000016,0.002634
create,6.192011e-04,3.127773e-05,-0.008519,0.049054,0.002001,1.412211e-03,1.000000e+00,0.004275,0.070016,0.140350,5.411898e-07,0.070909,1.129563e-03,6.909923e-03,9.363771e-03,0.039349,0.013733,0.014638,0.082203,1.280771e-03,-0.001251,0.034942,5.770353e-04,0.049259,0.038218,4.481873e-03,5.817517e-04,0.029844,1.118585e-03,0.074238,0.021854,0.000934,0.000831,1.525189e-03,1.129563e-03,1.129563e-03,0.148453,0.015174,5.411898e-07,0.010879,0.006189,0.002748,0.003136,7.465019e-04,0.000966,0.016388
create2,5.12524

In [16]:
respuesta_7['cantidad_de_bloqueos'].nlargest(5)

cantidad_de_bloqueos    1.000000
create                  0.000966
deleteuser             -0.000013
undelete               -0.000013
usergroups             -0.000013
Name: cantidad_de_bloqueos, dtype: float64

In [17]:
respuesta_7['cantidad_de_bloqueos'].argmax()

44

In [18]:
respuesta_7['ingles'].nlargest(5)

ingles          1.000000
thank           0.070853
move_redir      0.067974
delete_redir    0.050484
upload          0.044374
Name: ingles, dtype: float64

In [19]:
respuesta_7['ingles'].argmax()

45